# Reinforcement learning (RL)
Goal of the lab:
- What is RL
    - Motivation
    - distinction between RL and supervised learning
    - distinction between RL and online learning
- Several important examples
    - Bandit problem
    
    - a control game
- An example using neural net to 
